In [3]:
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
import json

In [ ]:
conn = None
# Cargar credenciales desde el archivo JSON
try:
    with open("../credenciales.json", "r") as file:
        credenciales = json.load(file)
except Exception as e:
    print(f"Error al cargar el archivo JSON: {e}")
    credenciales = None

# Establecer la conexión
if credenciales:  # Solo intentar conectar si las credenciales se cargaron correctamente
    try:
        conn = psycopg2.connect(
            user=credenciales["user"],
            password=credenciales["password"],
            host=credenciales["host"],
            port=credenciales["port"],
            database=credenciales["database"]
        )
        print("¡Conexión exitosa!")
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
else:
    print("No se pudieron cargar las credenciales. No se puede establecer la conexión.")

¡Conexión exitosa!


In [10]:
#if conn:
#    conn.close()
#    print("Conexión cerrada correctamente.")
#else:
#    print("No hay una conexión activa para cerrar.")

In [ ]:
# Cargar credenciales desde el archivo JSON
try:
    with open("../credenciales.json", "r") as file:
        credenciales = json.load(file)
except Exception as e:
    print(f"Error al cargar el archivo JSON: {e}")
    credenciales = None

# Crear la cadena de conexión
if credenciales:
    try:
        connection_string = f"postgresql+psycopg2://{credenciales['user']}:{credenciales['password']}@{credenciales['host']}:{credenciales['port']}/{credenciales['database']}"
        engine = create_engine(connection_string)
        print("¡Conexión exitosa con SQLAlchemy!")
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
else:
    print("No se pudieron cargar las credenciales. No se puede establecer la conexión.")

¡Conexión exitosa con SQLAlchemy!


In [26]:
df = pd.read_csv("../data/candidates.csv", delimiter=';')

# Mostrar las primeras filas para verificar
print(df.head())

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

In [ ]:
table_name = 'candidates'
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    "First Name" TEXT,
    "Last Name" TEXT,
    "Email" TEXT,
    "Application Date" DATE,
    "Country" TEXT,
    "YOE" INTEGER,
    "Seniority" TEXT,
    "Technology" TEXT,
    "Code Challenge Score" FLOAT,
    "Technical Interview Score" FLOAT
);
"""

try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
        connection.commit()  # Confirmar la transacción
        print(f"¡Tabla {table_name} creada exitosamente!")
except Exception as e:
    print(f"Error al crear la tabla: {e}")

¡Tabla candidates creada exitosamente!


In [20]:
# Cargar el DataFrame en la tabla de PostgreSQL
try:
    df.to_sql(table_name, engine, if_exists='append', index=False)
    print(f"¡Datos cargados exitosamente en la tabla {table_name}!")
except Exception as e:
    print(f"Error al cargar los datos en la base de datos: {e}")

¡Datos cargados exitosamente en la tabla candidates!


In [25]:
total_filas_csv = len(df)
print(f"El archivo CSV tiene un total de {total_filas_csv} filas.")

try:
    with engine.connect() as connection:
        # Ejecutar la consulta
        result = connection.execute(text(f"SELECT COUNT(*) FROM {table_name}"))
        total_registros = result.scalar()  # Obtener el resultado de la consulta
        print(f"El número total de registros en la tabla {table_name} es: {total_registros}")
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")

El archivo CSV tiene un total de 50000 filas.
El número total de registros en la tabla candidates es: 50000
